In [1]:
import os
execfile(os.path.join(os.environ["SPARK_HOME"], 'python/pyspark/shell.py'))

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.1.1
      /_/

Using Python version 2.7.12 (default, Nov 19 2016 06:48:10)
SparkSession available as 'spark'.


In [2]:
from pyspark.sql import SparkSession
sparkSession = SparkSession.builder.enableHiveSupport().master("local [2]").getOrCreate()

In [3]:
data = sparkSession.read.parquet("/data/sample264")
meta = sparkSession.read.parquet("/data/meta")

## Normalization could be done by next function

In [4]:
from pyspark.sql import Window
from pyspark.sql.functions import row_number, sum

def norm(df, key1, key2, field, n): 
    
    window = Window.partitionBy(key1).orderBy(col(field).desc())
        
    topsDF = df.withColumn("row_number", row_number().over(window)) \
        .filter(col("row_number") <= n) \
        .drop(col("row_number")) 
        
    tmpDF = topsDF.groupBy(col(key1)).agg(col(key1), sum(col(field)).alias("sum_" + field))
   
    normalizedDF = topsDF.join(tmpDF, key1, "inner") \
        .withColumn("norm_" + field, col(field) / col("sum_" + field)) \
        .cache()

    return normalizedDF

In [5]:
from pyspark.sql import Window
from pyspark.sql.functions import col, rank

userTrack = data.groupBy(col("userId"), col("trackId")).count()

userTrackNorm = norm(userTrack, "userId", "trackId", "count", 1000) \
        .withColumn("id", col("userId")) \
        .withColumn("id2", col("trackId")) \
        .withColumn("norm_count", col("norm_count") * 0.5) \
        .select(col("id"), col("id2"), col("norm_count"))     

window = Window.orderBy(col("norm_count"))
    
userTrackList = userTrackNorm.withColumn("position", rank().over(window))\
    .filter(col("position") < 50)\
    .orderBy(col("id"), col("id2"))\
    .select(col("id"), col("id2"))\
    .take(40)

# Task 1

In [6]:
from pyspark.sql.functions import col, desc, asc, collect_list

In [7]:
SEVEN_MIN = 60 * 7

tracks_tracks = data \
    .alias('data_1').join(data.alias('data_2'), \
        (col('data_1.userId') == col('data_2.userId')) &
        (col('data_1.trackId') != col('data_2.trackId')) &
        (
            (col('data_1.timestamp') - col('data_2.timestamp') <= SEVEN_MIN) &
            (col('data_1.timestamp') - col('data_2.timestamp') >= -SEVEN_MIN)
        ), 'inner') \
    .select(
        col('data_1.trackId').alias('id1'),
        col('data_2.trackId').alias('id2')
    ) \
    .groupBy(col('id1'), col('id2')).count() \
    .orderBy(desc('count'))

tracks_tracks.show(5)

+------+------+-----+
|   id1|   id2|count|
+------+------+-----+
|870292|939606|  253|
|939606|870292|  253|
|854531|879259|  195|
|879259|854531|  195|
|933030|871513|  159|
+------+------+-----+
only showing top 5 rows



In [8]:
results = norm(tracks_tracks, 'id1', 'id2', 'count', 40) \
    .select(col('id1'), col('id2'), col('norm_count')) \
    .orderBy(desc('norm_count'), asc('id1'), asc('id2'))

results.show(5)

+------+------+----------+
|   id1|   id2|norm_count|
+------+------+----------+
|798256|923706|       1.0|
|798319|837992|       1.0|
|798322|876562|       1.0|
|798331|827364|       1.0|
|798335|840741|       1.0|
+------+------+----------+
only showing top 5 rows



In [9]:
for row in results.take(40):
    print row['id1'], row['id2']

798256 923706
798319 837992
798322 876562
798331 827364
798335 840741
798374 816874
798375 810685
798379 812055
798380 840113
798396 817687
798398 926302
798405 867217
798443 905923
798457 918918
798460 891840
798461 940379
798470 840814
798474 963162
798477 883244
798485 955521
798505 905671
798545 949238
798550 936295
798626 845438
798691 818279
798692 898823
798702 811440
798704 937570
798725 933147
798738 894170
798745 799665
798782 956938
798801 950802
798820 890393
798833 916319
798865 962662
798931 893574
798946 946408
799012 809997
799024 935246


# Task 2

In [10]:
users_tracks = data \
    .select(
        col('userId').alias('id1'),
        col('trackId').alias('id2')) \
    .groupBy('id1', 'id2').count() \
    .orderBy(desc('count'), asc('id1'), asc('id2'))

users_tracks.show(5)

+------+------+-----+
|   id1|   id2|count|
+------+------+-----+
|668849|817132|  277|
|560428|950984|   94|
|767478|870292|   94|
|278647|940362|   87|
|770607|830615|   76|
+------+------+-----+
only showing top 5 rows



In [11]:
results = norm(users_tracks, 'id1', 'id2', 'count', 1000) \
    .select(col('id1'), col('id2'), col('norm_count')) \
    .orderBy(desc('norm_count'), asc('id1'), asc('id2'))

results.show(5)

+---+------+----------+
|id1|   id2|norm_count|
+---+------+----------+
| 66|965774|       1.0|
|116|867268|       1.0|
|128|852564|       1.0|
|131|880170|       1.0|
|195|946408|       1.0|
+---+------+----------+
only showing top 5 rows



In [12]:
for row in results.take(40):
    print row['id1'], row['id2']

66 965774
116 867268
128 852564
131 880170
195 946408
215 860111
235 897176
300 857973
321 915545
328 943482
333 818202
346 864911
356 961308
428 943572
431 902497
445 831381
488 841340
542 815388
617 946395
649 901672
658 937522
662 881433
698 935934
708 952432
746 879259
747 879259
776 946408
784 806468
806 866581
811 948017
837 799685
901 871513
923 879322
934 940714
957 945183
989 878364
999 967768
1006 962774
1049 849484
1057 920458


# Task 3

In [13]:
users_artists = data \
    .select(
        col('userId').alias('id1'),
        col('artistId').alias('id2')) \
    .groupBy('id1', 'id2').count() \
    .orderBy(desc('count'), asc('id1'), asc('id2'))

users_artists.show(5)

+------+-------+-----+
|   id1|    id2|count|
+------+-------+-----+
|668849| 994686|  277|
|436158|1003021|  142|
|442306|1001300|  107|
|560428| 975695|   94|
|767478| 991179|   94|
+------+-------+-----+
only showing top 5 rows



In [14]:
results = norm(users_artists, 'id1', 'id2', 'count', 100) \
    .select(col('id1'), col('id2'), col('norm_count')) \
    .orderBy(desc('norm_count'), asc('id1'), asc('id2'))

results.show(5)

+---+-------+----------+
|id1|    id2|norm_count|
+---+-------+----------+
| 66| 993426|       1.0|
|116| 974937|       1.0|
|128|1003021|       1.0|
|131| 983068|       1.0|
|195| 997265|       1.0|
+---+-------+----------+
only showing top 5 rows



In [15]:
for row in results.take(40):
    print row['id1'], row['id2']

66 993426
116 974937
128 1003021
131 983068
195 997265
215 991696
235 990642
288 1000564
300 1003362
321 986172
328 967986
333 1000416
346 982037
356 974846
374 1003167
428 993161
431 969340
445 970387
488 970525
542 969751
612 987351
617 970240
649 973851
658 973232
662 975279
698 995788
708 968848
746 972032
747 972032
776 997265
784 969853
806 995126
811 996436
837 989262
901 988199
923 977066
934 990860
957 991171
989 975339
999 968823


# Task 4

In [16]:
artists_tracks = data \
    .select(
        col('artistId').alias('id1'),
        col('trackId').alias('id2')) \
    .groupBy('id1', 'id2').count() \
    .orderBy(desc('count'), asc('id1'), asc('id2'))

artists_tracks.show(5)

+------+------+-----+
|   id1|   id2|count|
+------+------+-----+
|987351|886091| 2958|
|988199|871513| 2904|
|997265|946408| 2836|
|981306|864690| 2582|
|974503|858904| 2453|
+------+------+-----+
only showing top 5 rows



In [17]:
results = norm(artists_tracks, 'id1', 'id2', 'count', 100) \
    .select(col('id1'), col('id2'), col('norm_count')) \
    .orderBy(desc('norm_count'), asc('id1'), asc('id2'))

results.show(5)

+------+------+----------+
|   id1|   id2|norm_count|
+------+------+----------+
|967993|869415|       1.0|
|967998|947428|       1.0|
|968004|927380|       1.0|
|968017|859321|       1.0|
|968022|852786|       1.0|
+------+------+----------+
only showing top 5 rows



In [18]:
for row in results.take(40):
    print row['id1'], row['id2']

967993 869415
967998 947428
968004 927380
968017 859321
968022 852786
968034 807671
968038 964150
968042 835935
968043 913568
968046 935077
968047 806127
968065 907906
968073 964586
968086 813446
968092 837129
968118 914441
968125 821410
968140 953008
968148 877445
968161 809793
968163 803065
968168 876119
968189 858639
968221 896937
968224 892880
968232 825536
968237 932845
968238 939177
968241 879045
968242 911250
968248 953554
968255 808494
968259 880230
968265 950148
968266 824437
968269 913243
968272 816049
968278 946743
968285 847460
968286 940006
